In [394]:
import pandas as pd
import re

In [395]:
    
def match_names(df):
    df = df.astype(str)
    df["names"] = " ".join(row for row in df["data"])
    names = df["names"][0]
    df = df.drop_duplicates(subset=["names"])

    NAME1 = r"[A-Z][a-z]+,?\s+"
    NAME2 = r"[A-Z][a-z]+\s+"

    result = re.findall(NAME1 + NAME2, names)
    df = pd.DataFrame(result, columns=["name"])
    return df
    
def read_txt_f_1():
    df = pd.read_csv(
        ("../data/MA/misconduct/nacdl/Newton/IIU/cprr_newton_2010_2021.pdf.txt"),
        sep="delimiter",
        header=None,
        engine="python",
        index_col=False,
    )
    df = (
        df\
        .rename(columns={df.columns[0]: "data"})
        .pipe(match_names)
    )
    return df

def match_titles(df):
    df = df.astype(str)
    ID = r"([Oo]fficer\(?s?\)?:?.+|[Ss]gt.+|[Ll]t.+|[Ddet].+|[Cc]a?pt.+|[Oo]fc.+|[Ss]ergeant.+|[Llieutenant].+|[Dd]etective.+)"

    result = [re.findall(ID, row) for row in df["data"]]
    df = pd.DataFrame(result, columns=["name"])
    df = df[~((df.name.fillna("") == ""))]


    return df

def read_txt_f_2():
    df = pd.read_csv(
        ("../data/MA/misconduct/nacdl/Newton/IIU/cprr_newton_2010_2021.pdf.txt"),
        sep="delimiter",
        header=None,
        engine="python",
        index_col=False,
    )
    df = (
        df\
        .rename(columns={df.columns[0]: "data"})
        .pipe(match_titles)
    )
    return df


In [396]:
df1 = read_txt_f_1()
df2 = read_txt_f_2()

In [397]:
def split_names(df):
    names = df.name.str.extract(r"(\w+) (\w+)")
    df.loc[:, "first_name"] = names[0]
    df.loc[:, "last_name"] = names[1]
    return df.drop(columns=["name"])

df1 = df1.pipe(split_names)
df1

,first_name,last_name
0,Officer,Behavior
1,Sustained,Exonerated
2,Sullivan,Exonerated
3,Babcock,Exonerated
4,Cottens,Exonerated
...,...,...
249,Dawn,Fleming
250,Chris,Howes
251,Lauren,Keefe
252,James,Crowe


In [398]:
def extract_names(df):
    names = df.name.str.extract(r"([Oo]ff?i?c?e?r?\(?s?\)?:?|[Ss]gt|[Ll]t|[Ddet]|[Cc]a?pt|[Ss]ergeant|[Ll]ieutenant|[Dd]etective)\.? (\w+) ?(\w+)?")
    df.loc[:, "rank_desc"] = names[0]
    df.loc[:, "first_name"] = names[1]
    df.loc[:, "last_name"] = names[2]
    return df 

df2 = df2.pipe(extract_names)
df2 = df2.drop(columns=["name", "rank_desc"])

df = pd.concat([df1, df2], axis=0)
df

,first_name,last_name
0,Officer,Behavior
1,Sustained,Exonerated
2,Sullivan,Exonerated
3,Babcock,Exonerated
4,Cottens,Exonerated
...,...,...
420,her,NaN
421,to,cause
423,reports,SIF
424,adds,name


In [399]:
# df.to_csv("../data/MA/misconduct/nacdl/Newton/IIU/cprr_newton_2010_2021.csv", index=False)